In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px

In [10]:
data = pd.read_csv('data/final.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job title           211 non-null    object
 1   job_type            211 non-null    object
 2   company             211 non-null    object
 3   location            211 non-null    object
 4   Experience Needed:  211 non-null    object
 5   Career Level:       211 non-null    object
 6   Education Level:    211 non-null    object
 7   Salary:             211 non-null    object
 8   Job Categories:     211 non-null    object
 9   skills and tools    211 non-null    object
 10  Job                 211 non-null    object
 11  skills              211 non-null    object
dtypes: object(12)
memory usage: 19.9+ KB


In [11]:
data.head()

,job title,job_type,company,location,Experience Needed:,Career Level:,Education Level:,Salary:,Job Categories:,skills and tools,Job,skills
0,Senior AI Engineer,Full Time,Softec Technologies,"Maadi, Cairo",'3 to 5 years','Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Engineering - Tele...","['Communication skills', 'Development', 'Artif...",AI Engineer,"['Communication skills', 'Development', 'Artif..."
1,AI & Social Media Expert - CPM,Full Time,Etisalat Misr,"Cairo, Egypt",'More than 8 years','Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Marketing/PR/Adver...","['CRM', 'Computer Science', 'English']",AI Engineer,"['CRM', 'Computer Science', 'English']"
2,AI Software Engineer (Computer Vision Operatio...,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",'More than 5 years','Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Job is Remotely from our Office...","'Operations/Management', 'IT/Software Developm...","['Kubernetes', 'Linux OS', 'Docker', 'Artifici...",AI Engineer,"['Kubernetes', 'Linux OS', 'Docker', 'Artifici..."
3,AI/BI Intern,Internship,Easy System,"Cairo, Egypt",'Not Specified','Entry Level (Junior Level / Fresh Grad)','Not Specified','paid',"'IT/Software Development', 'Engineering - Tele...","['Artificial Intelligence', 'Artificial Intell...",AI Engineer,"['Artificial Intelligence', 'Artificial Intell..."
4,Embedded Software Engineer,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",'More than 2 years','Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Working remotely with Abu Dhabi...","'IT/Software Development', 'Engineering - Tele...","['Embedded System', 'Development', 'Linux OS',...",AI Engineer,"['Embedded System', 'Development', 'Linux OS',..."


# 1- Handle Experience ranges

In [12]:
def mapper(x):
    x = x.replace(':','')
    x = x.lower()
    x = x.replace(' ', '_')
    return x
data.rename(columns = mapper, inplace = True)

In [13]:
def ranging(num):
    num = int(num)
    if num <= 2:
        return '0 to 2 years'
    elif num <= 4:
        return '2 to 4 years'
    elif num  <= 6: 
        return '4 to 6 years'
    elif num  <= 8:
        return '6 to 8 years'
    elif num  <= 10:
        return '8 to 10 years'
    elif num  > 10:
            return 'more than 10 years'

def exp_ranging(x):
    try :
        temp = list(map(str.strip,x.split('to')))
        return ranging(temp[1][0:2].strip())
    except:
        temp = x.split()
        if temp[0].lower() == 'not':
            return 'Internship' # no mean of experience internship so handle it 
        elif temp[0].lower() == 'more':
            # return '> {}'.format(temp[2])
            return ranging(int(temp[2][0:2].strip()) + 1)
        else:
            return ranging(temp[0])






In [14]:
# extracing the enclose string from experience needed column data before like this --> "'1 to 3 years'"
data['experience_needed'] = data['experience_needed'].apply(lambda x: x.replace("'",''))

In [15]:
data['exp_ranges2'] = data['experience_needed'].apply(exp_ranging)
data.head()

,job_title,job_type,company,location,experience_needed,career_level,education_level,salary,job_categories,skills_and_tools,job,skills,exp_ranges2
0,Senior AI Engineer,Full Time,Softec Technologies,"Maadi, Cairo",3 to 5 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Engineering - Tele...","['Communication skills', 'Development', 'Artif...",AI Engineer,"['Communication skills', 'Development', 'Artif...",4 to 6 years
1,AI & Social Media Expert - CPM,Full Time,Etisalat Misr,"Cairo, Egypt",More than 8 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Marketing/PR/Adver...","['CRM', 'Computer Science', 'English']",AI Engineer,"['CRM', 'Computer Science', 'English']",8 to 10 years
2,AI Software Engineer (Computer Vision Operatio...,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",More than 5 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Job is Remotely from our Office...","'Operations/Management', 'IT/Software Developm...","['Kubernetes', 'Linux OS', 'Docker', 'Artifici...",AI Engineer,"['Kubernetes', 'Linux OS', 'Docker', 'Artifici...",4 to 6 years
3,AI/BI Intern,Internship,Easy System,"Cairo, Egypt",Not Specified,'Entry Level (Junior Level / Fresh Grad)','Not Specified','paid',"'IT/Software Development', 'Engineering - Tele...","['Artificial Intelligence', 'Artificial Intell...",AI Engineer,"['Artificial Intelligence', 'Artificial Intell...",Internship
4,Embedded Software Engineer,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",More than 2 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Working remotely with Abu Dhabi...","'IT/Software Development', 'Engineering - Tele...","['Embedded System', 'Development', 'Linux OS',...",AI Engineer,"['Embedded System', 'Development', 'Linux OS',...",2 to 4 years


In [16]:
# rearranging the columns for better inspection
data.iloc[:,4]
# data.drop('exp_ranges', axis= 1, inplace=True)
poped = data.pop('exp_ranges2')
data.insert(loc= 5, column='exp_ranges', value=poped)

data.head()

,job_title,job_type,company,location,experience_needed,exp_ranges,career_level,education_level,salary,job_categories,skills_and_tools,job,skills
0,Senior AI Engineer,Full Time,Softec Technologies,"Maadi, Cairo",3 to 5 years,4 to 6 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Engineering - Tele...","['Communication skills', 'Development', 'Artif...",AI Engineer,"['Communication skills', 'Development', 'Artif..."
1,AI & Social Media Expert - CPM,Full Time,Etisalat Misr,"Cairo, Egypt",More than 8 years,8 to 10 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Marketing/PR/Adver...","['CRM', 'Computer Science', 'English']",AI Engineer,"['CRM', 'Computer Science', 'English']"
2,AI Software Engineer (Computer Vision Operatio...,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",More than 5 years,4 to 6 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Job is Remotely from our Office...","'Operations/Management', 'IT/Software Developm...","['Kubernetes', 'Linux OS', 'Docker', 'Artifici...",AI Engineer,"['Kubernetes', 'Linux OS', 'Docker', 'Artifici..."
3,AI/BI Intern,Internship,Easy System,"Cairo, Egypt",Not Specified,Internship,'Entry Level (Junior Level / Fresh Grad)','Not Specified','paid',"'IT/Software Development', 'Engineering - Tele...","['Artificial Intelligence', 'Artificial Intell...",AI Engineer,"['Artificial Intelligence', 'Artificial Intell..."
4,Embedded Software Engineer,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",More than 2 years,2 to 4 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Working remotely with Abu Dhabi...","'IT/Software Development', 'Engineering - Tele...","['Embedded System', 'Development', 'Linux OS',...",AI Engineer,"['Embedded System', 'Development', 'Linux OS',..."


In [17]:
data.drop('experience_needed', axis=1, inplace=True)

In [18]:
data.head(5)

,job_title,job_type,company,location,exp_ranges,career_level,education_level,salary,job_categories,skills_and_tools,job,skills
0,Senior AI Engineer,Full Time,Softec Technologies,"Maadi, Cairo",4 to 6 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Engineering - Tele...","['Communication skills', 'Development', 'Artif...",AI Engineer,"['Communication skills', 'Development', 'Artif..."
1,AI & Social Media Expert - CPM,Full Time,Etisalat Misr,"Cairo, Egypt",8 to 10 years,'Experienced (Non-Manager)',"""Bachelor's Degree""",'Confidential',"'IT/Software Development', 'Marketing/PR/Adver...","['CRM', 'Computer Science', 'English']",AI Engineer,"['CRM', 'Computer Science', 'English']"
2,AI Software Engineer (Computer Vision Operatio...,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",4 to 6 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Job is Remotely from our Office...","'Operations/Management', 'IT/Software Developm...","['Kubernetes', 'Linux OS', 'Docker', 'Artifici...",AI Engineer,"['Kubernetes', 'Linux OS', 'Docker', 'Artifici..."
3,AI/BI Intern,Internship,Easy System,"Cairo, Egypt",Internship,'Entry Level (Junior Level / Fresh Grad)','Not Specified','paid',"'IT/Software Development', 'Engineering - Tele...","['Artificial Intelligence', 'Artificial Intell...",AI Engineer,"['Artificial Intelligence', 'Artificial Intell..."
4,Embedded Software Engineer,Full Time,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt",2 to 4 years,'Experienced (Non-Manager)',"""Bachelor's Degree""","'Confidential, Working remotely with Abu Dhabi...","'IT/Software Development', 'Engineering - Tele...","['Embedded System', 'Development', 'Linux OS',...",AI Engineer,"['Embedded System', 'Development', 'Linux OS',..."


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         211 non-null    object
 1   job_type          211 non-null    object
 2   company           211 non-null    object
 3   location          211 non-null    object
 4   exp_ranges        211 non-null    object
 5   career_level      211 non-null    object
 6   education_level   211 non-null    object
 7   salary            211 non-null    object
 8   job_categories    211 non-null    object
 9   skills_and_tools  211 non-null    object
 10  job               211 non-null    object
 11  skills            211 non-null    object
dtypes: object(12)
memory usage: 19.9+ KB


In [20]:
# Handle skills

# 2. visualize

In [21]:
def grouping_data(columns_to_group_by, selected_column):
    groups_count=data.groupby(columns_to_group_by)[selected_column].count()
    values=groups_count.values
    column1=[]
    column2=[]
    for c in groups_count.index:
        column1.append(c[0])
        column2.append(c[1])

    df=pd.DataFrame(data={columns_to_group_by[0]:column1,
                   selected_column: column2,
                            'count':values
                           } )
   
    return df

    

## 2.1 experience ranges

In [22]:
exp_ranges = ['Internship','0 to 2 years', '2 to 4 years', '4 to 6 years', '6 to 8 years','8 to 10 years', 'more than 10 years']

In [23]:
# data['exp_ranges'].value_counts()

In [24]:
exp_df=grouping_data(['job','exp_ranges'],'exp_ranges') 

exp_df.head()

,job,exp_ranges,count
0,AI Engineer,0 to 2 years,8
1,AI Engineer,2 to 4 years,17
2,AI Engineer,4 to 6 years,12
3,AI Engineer,6 to 8 years,3
4,AI Engineer,8 to 10 years,6


In [299]:
experience_fig=px.bar(exp_df , y='exp_ranges', x='count' ,
       color='job',color_discrete_sequence=px.colors.qualitative.T10 , 
       category_orders={'x': exp_ranges}  )
experience_fig.update_layout(yaxis={'categoryorder':'total ascending'})
# experience_fig.update_layout(title_text='Data Jobs Experience Ranges' ,title_x=0.5)

## 2.2 job type

In [26]:
job_type_df=grouping_data(['job','job_type'],'job_type')  
job_type_df.head()

,job,job_type,count
0,AI Engineer,Freelance / Project,2
1,AI Engineer,Full Time,42
2,AI Engineer,Internship,5
3,AI Engineer,Part Time,2
4,Data Analyst,Freelance / Project,1


In [309]:
job_type_bar=px.bar(job_type_df,  y='job_type' ,x='count', color='job',color_discrete_sequence=px.colors.qualitative.T10, )
job_type_bar.update_layout(yaxis={'categoryorder':'total ascending'})

## 2.3 career level

In [28]:
career_level_df=grouping_data(['job','career_level'],'career_level')  
career_level_df.head()

,job,career_level,count
0,AI Engineer,'Entry Level (Junior Level / Fresh Grad)',20
1,AI Engineer,'Experienced (Non-Manager)',26
2,AI Engineer,'Manager ',4
3,AI Engineer,'Student (Undergrad / Postgrad)',1
4,Data Analyst,'Entry Level (Junior Level / Fresh Grad)',14


In [296]:
career_level_bar=px.bar(career_level_df,y= 'career_level' , x='count' , color='job' , color_discrete_sequence=px.colors.qualitative.T10)
career_level_bar.update_layout(yaxis={'categoryorder':'total ascending'})


In [30]:
career_level_pie=px.pie(career_level_df , names='career_level' , values='count',
                        color_discrete_sequence=px.colors.qualitative.T10 ,
                       title='experience and career level')
career_level_pie.update_layout(title_x=0.5)

career_level_pie

In [31]:
career_level_df['job'].unique()

array(['AI Engineer', 'Data Analyst', 'Data Engineer', 'Data scientist',
       'ML Engineer'], dtype=object)

In [32]:
# career level pie for each job 
px.pie(career_level_df[career_level_df['job']=='Data Engineer'] , names='career_level' , values='count',color_discrete_sequence=px.colors.qualitative.T10)

In [33]:
# bar chart for experience needed and career level

exp_career_df=grouping_data(['exp_ranges','career_level'],'career_level')
exp_career_df.head()

,exp_ranges,career_level,count
0,0 to 2 years,'Entry Level (Junior Level / Fresh Grad)',26
1,0 to 2 years,'Experienced (Non-Manager)',1
2,0 to 2 years,'Student (Undergrad / Postgrad)',1
3,2 to 4 years,'Entry Level (Junior Level / Fresh Grad)',28
4,2 to 4 years,'Experienced (Non-Manager)',33


In [300]:
career_experience_bar=px.bar(exp_career_df, y='exp_ranges', x='count', color='career_level', color_discrete_sequence=px.colors.qualitative.T10)
career_experience_bar.update_layout(yaxis={'categoryorder':'total ascending'})

## 2.4 companies

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         211 non-null    object
 1   job_type          211 non-null    object
 2   company           211 non-null    object
 3   location          211 non-null    object
 4   exp_ranges        211 non-null    object
 5   career_level      211 non-null    object
 6   education_level   211 non-null    object
 7   salary            211 non-null    object
 8   job_categories    211 non-null    object
 9   skills_and_tools  211 non-null    object
 10  job               211 non-null    object
 11  skills            211 non-null    object
dtypes: object(12)
memory usage: 19.9+ KB


In [36]:
companies_df=grouping_data(['job','company'],'company')
companies_df

,job,company,count
0,AI Engineer,360Imaging,2
1,AI Engineer,AFKAR Technology,1
2,AI Engineer,Atomica,11
3,AI Engineer,Beyond Apps Group,1
4,AI Engineer,Easy System,1
...,...,...,...
159,ML Engineer,34ML,3
160,ML Engineer,Corporatica,1
161,ML Engineer,Tahaluf Al Emarat Technical Solutions,2
162,ML Engineer,Talents Arena,1


In [37]:
companies_bar=px.bar(companies_df[companies_df['count']>1], y='company', x='count', 
                     color='job', color_discrete_sequence=px.colors.qualitative.T10,
                    title='Top Hiring Companies')

companies_bar.update_layout(yaxis={'categoryorder':'total ascending'},title_x=0.5)


In [38]:
px.scatter(companies_df[companies_df['count']>=2],y='company',size='count', color='job',color_discrete_sequence=px.colors.qualitative.T10 )

In [39]:
px.scatter(companies_df[companies_df['count']>=2],y='company',x='count',size='count', color='job',color_discrete_sequence=px.colors.qualitative.T10 )

## 2.5 Locations

In [40]:
data[['job','company','location']]

,job,company,location
0,AI Engineer,Softec Technologies,"Maadi, Cairo"
1,AI Engineer,Etisalat Misr,"Cairo, Egypt"
2,AI Engineer,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt"
3,AI Engineer,Easy System,"Cairo, Egypt"
4,AI Engineer,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt"
...,...,...,...
206,Data scientist,Standout Solutions,"New Cairo, Cairo"
207,Data scientist,Mobile Wave 3,"Nasr City, Cairo"
208,Data scientist,Squadio,"Cairo, Egypt"
209,Data scientist,Tahaluf Al Emarat Technical Solutions,"Cairo, Egypt"


In [41]:
data['location'].unique()

array(['Maadi, Cairo', 'Cairo, Egypt', 'Nasr City, Cairo',
       'Riyadh, Saudi Arabia', 'New Delhi, India', 'Giza, Egypt',
       '6th of October, Giza', 'Mohandessin, Giza', 'Dokki, Giza',
       'Bilbao, Spain', 'New Cairo, Cairo', 'Heliopolis, Cairo',
       'Sheraton, Cairo', 'Beirut, Lebanon', 'Smart Village, Giza',
       '10th of Ramadan City, Cairo', 'Banha, Qalubia',
       'Sheikh Zayed, Giza', 'Moharam Bek, Alexandria',
       'Abu Rawash, Giza', 'Downtown, Cairo', 'Jizan, Saudi Arabia',
       'Imbaba, Giza', 'Giza, Giza', 'Cairo, Egypt Not Verified',
       'Abu Dhabi, United Arab Emirates', 'Stanley, Alexandria',
       'Salam City, Cairo', '10th of Ramadan City, Sharqia',
       'Helwan, Cairo', 'Mahalla Kubra, Gharbia',
       'San Stefano, Alexandria', 'Suez, Egypt', 'Badr City, Cairo',
       'Bourj Alarab, Alexandria', 'North Coast, Matruh',
       'Ain Sokhna, Suez', 'Mansoura, Dakahlia'], dtype=object)

In [42]:
#map

## 2.5 Skills

### Factorize to calculate frequency of skills

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer

In [230]:
ROLE_COLS      = ['skills']
def one_hot(job_df ):
    encoded_dfs = {}
    for col in ROLE_COLS :
        binarizer = MultiLabelBinarizer()
        encoded_df = pd.DataFrame(binarizer.fit_transform(job_df[col]),
                                   columns=binarizer.classes_,
                                   index=job_df[col].index)
        encoded_dfs[col] = encoded_df
    # Merge 1-hot encoded 
    skills_df = pd.concat(encoded_dfs, axis=1)
    return skills_df
    

### 2.5.1 AI Engineer

In [231]:
df=data.copy()
df['skills'].iloc[0]

"['Communication skills', 'Development', 'Artificial Intelligence', 'Artificial Intelligence', 'Computer Science', 'English']"

In [232]:
#remove double quotes from skills data
# remove double quotes 

df['skills']=df['skills'].apply(lambda r: eval(r))



In [233]:
df['skills'].iloc[0]

['Communication skills',
 'Development',
 'Artificial Intelligence',
 'Artificial Intelligence',
 'Computer Science',
 'English']

In [234]:
AI_skills= one_hot(df[df['job']=='AI Engineer'] )
AI_skills_freq = AI_skills.sum().reset_index()
AI_skills_freq.columns = ['group', 'skill', 'freq']
AI_skills_freq.head() 

,group,skill,freq
0,skills,API,4
1,skills,Agile,7
2,skills,Algorithm,5
3,skills,Amazon Web Services,1
4,skills,Amelia Plugin,1


In [254]:
AI_skills_fig=px.scatter(AI_skills_freq[AI_skills_freq['freq']>5], y='skill', x='freq', size='freq',color='skill',
                        title='AI Engineer Skills and Tools')
AI_skills_fig=AI_skills_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5)

### 2.5.2 ML Engineer

In [253]:
ML_skills= one_hot(df[df['job']=='ML Engineer'] )
ML_skills_freq = ML_skills.sum().reset_index()
ML_skills_freq.columns = ['group', 'skill', 'freq']
ML_skills_freq.head() 

,group,skill,freq
0,skills,API,1
1,skills,Algorithm,3
2,skills,Amazon Web Services,2
3,skills,Angular,2
4,skills,Azure DevOps Server,1


In [255]:
ML_skills_fig=px.scatter(ML_skills_freq, y='skill', x='freq', size='freq',color='skill',
                         title='ML Engineer Skills and Tools')
ML_skills_fig=ML_skills_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5)


### 2.5.3 Data scientist

In [238]:
DS_skills= one_hot(df[df['job']=='Data scientist'] )
DS_skills_freq = DS_skills.sum().reset_index()
DS_skills_freq.columns = ['group', 'skill', 'freq']
DS_skills_freq.head() 

,group,skill,freq
0,skills,.Net,3
1,skills,API,1
2,skills,Agile,2
3,skills,Analytical Skills,1
4,skills,Angular,7


In [256]:
DS_skills_fig=px.scatter(DS_skills_freq, y='skill', x='freq', size='freq',color='skill',
                        title='DS Engineer Skills and Tools')
DS_skills_fig=DS_skills_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5
                    )

### 2.5.4 Data Analyst

In [240]:
DA_skills= one_hot(df[df['job']=='Data Analyst'] )
DA_skills_freq = DA_skills.sum().reset_index()
DA_skills_freq.columns = ['group', 'skill', 'freq']
DA_skills_freq.head() 

,group,skill,freq
0,skills,Agile,2
1,skills,Amazon Web Services,2
2,skills,Analytical Skills,52
3,skills,BRD,1
4,skills,Big Data,1


In [257]:
DA_skill_fig=px.scatter(DA_skills_freq, y='skill', x='freq', size='freq',color='skill',
                       title='DA Engineer Skills and Tools')
DA_skill_fig=DA_skill_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5
                    )

### 2.5.5 Data Engineer

In [258]:
DE_skills= one_hot(df[df['job']=='Data scientist'] )
DE_skills_freq = DE_skills.sum().reset_index()
DE_skills_freq.columns = ['group', 'skill', 'freq']
DE_skills_freq.head() 

,group,skill,freq
0,skills,.Net,3
1,skills,API,1
2,skills,Agile,2
3,skills,Analytical Skills,1
4,skills,Angular,7


In [259]:
DE_skills_fig=px.scatter(DE_skills_freq, y='skill', x='freq', size='freq',color='skill',
                        title='DE Engineer Skills and Tools')
DE_skills_fig=DE_skills_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5
                    )

### 2.5.5 ALL skills

In [244]:
total_skills= one_hot(df )
total_skills_freq = total_skills.sum().reset_index()
total_skills_freq.columns = ['group', 'skill', 'freq']
total_skills_freq.head() 

,group,skill,freq
0,skills,.Net,3
1,skills,API,7
2,skills,Admin,3
3,skills,Agile,14
4,skills,Algorithm,10


In [261]:
skills_fig=px.scatter(total_skills_freq[total_skills_freq['freq']>5], y='skill', x='freq', size='freq',color='skill',
                     title='AI and Data Skills and Tools')
skills_fig=skills_fig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                        title_x=0.5
                    )

## 2.6 time series data

## 2.7 Salaries

In [222]:
salaries_df=data[['job','company','salary']]




In [223]:
# data['salary'].unique()

In [45]:
salaries= ["'Confidential'",
       "'Confidential, Job is Remotely from our Office in Egypt'",
       "'paid'", "'Confidential, Working remotely with Abu Dhabi HQ'",
       "'Confidential, Commission, Transportation and Bonus are included'",
       "'Confidential, Social Insurance / Medical Insurance'",
       "'Confidential, Working remotely with us'",
       "'Confidential, Working Remotely'",
       "'Confidential, Bonus, overtime compensation, social insurance, medical insurance and salary raise'",
       "'Confidential, Hybrid Work Mode, Overtime Compensation, Social Insurance, Medical Insurance & Salary Raise'",
       "'Gender:', 'Male'",
       "'Confidential, Transportation , Social Insurance ,Medical Insurance'",
       "'Confidential, Social & medical insurance'",
       "'Confidential, Profit Share, Transportation, Medical Insurance, Social Insurance, Life Insurance And Retirement Plan'",
       "'Confidential, Bonus - Profit Share'",
       "'Confidential, Bonus, Profit Share'",
       "'Confidential, Quarter Profit Share + Annual Fixed Bonus'",
       "'Confidential, Bonus, Social Security, Medical Insurance'",
        "'unpaid'", 
       "'Confidential, Profit Share'", 
       "'Confidential, Bounce\\nCommission'",
       "'Confidential, Social Insurance, Medical Insurance & Semi-Annual Bonus'",
       "'Confidential, Medical ,Social & Life Insurance'",
        "'Confidential, 1000'",
       "'Confidential, Annual Based On Performance'",
       "'Confidential, Medical Insurance, Social Insurance, Life Insurance, Retirement Plan Transportation and Profit Share'",
       "'Confidential, + Competitive Package + Social Insurance + Friendly Working Environment'"]

In [47]:
# for i, r  in salaries_df.iterrows():
    
#     if r['salary'] in salaries:
#         salaries_df.drop(r)

In [ ]:
# rempve companies named confidential from the df

#job type
#career level
#locations on map
#ايه اكتر شركة في ال  salary ranges
#top hiring companies
#skills


# Dashboard

In [48]:
from dash import Dash,html,dcc

from dash.dependencies import Input, Output,State
import dash_bootstrap_components as dbc


In [301]:
# Draw figure Function

def drawFigure(figure,  config={'displayModeBar': False, 'autosizable':True, 'responsive':True}):
    return dcc.Graph(
        
                    figure=figure.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                    ),
                    config=config,
                    style={'background-color':'#32383E', 'height':'100%'}
                )

In [302]:
px.colors.qualitative.T10

['#4C78A8',
 '#F58518',
 '#E45756',
 '#72B7B2',
 '#54A24B',
 '#EECA3B',
 '#B279A2',
 '#FF9DA6',
 '#9D755D',
 '#BAB0AC']

In [303]:
app = Dash(
    external_stylesheets=[dbc.themes.SLATE]
)

app.layout=html.Div(children=[
    
    
    
    dbc.Card([ 
                
#           dbc.CardImg(src='/static/images/wuzzuf-logo.png', top=True),
              
          dbc.CardBody([

              ##########title row
              dbc.Row([
                  dbc.Col([
                      dbc.CardImg(src='/static/images/wuzzuf-logo.png', top=True)
                  ], width=2),

                  dbc.Col([
                      html.H1('AI and Data Jobs in Egypt' ),

                  ], width=6,style={'text-align': 'center' }),
                  #BANs
                  dbc.Col([
                      html.H2("65.4", style={'color':'#4C78A8' }),
                     
#                       dbc.Badge("65.4", text_color="primary"),
#                       dbc.Badge("1.42", text_color="primary"),
#                       dbc.Badge("4.74", text_color="primary"),
#                       dbc.Badge("25.4", text_color="primary"),
                  ]),
                  
                  dbc.Col([
                       html.H2("28.4", style={'color':'#F58518' }),
                      
                  ]),
                  dbc.Col([
                      html.H2("4.74", style={'color':'#E45756' }),
                      
                  ]),
                  dbc.Col([
                      html.H2("1.42", style={'color':'#72B7B2' }),
                  ])
                  
                  
              ] , align=  'center' ,style={ 'height':'3rem'}),
              
              
              
              
              
              ############add column for BANs      
        
        
        html.Br(),
        #--------------------------------2nd row Graphs----------------------------------------------#
        dbc.Row([
            
            dbc.Col([
                drawFigure( figure=companies_bar),
                
            ], style={'width':'3rem' , 'height':'25rem'}),
            
            
            dbc.Col([
                
#                 dbc.DropdownMenu(id='input', label="Skills and Tools", 
#                                  children=[
#                                      dbc.DropdownMenuItem( "Skills and Tools", id="all"),
#                 ]),
#                 drawFigure( figure=skills_fig)
                

            
             html.Div( [dcc.Dropdown(id='input',options = [
            {'label':'ALL SKills and Tools', 'value':'ALL' },
             {'label':'AI skills and Tools', 'value':'AI' },
             {'label':'ML skills and Tools', 'value':'ML' },
             {'label':'DS SKills and Tools', 'value':'DS' },
             {'label':'DA SKills and Tools', 'value':'DA' },
             {'label':'DE SKills and Tools', 'value':'DE' },],value = 'ALL' ,multi=False) ],
                      style={'textAlign':'center','width':200,'height':50, 
                             'background-color': '#32383E','color': '#212121'}), 
            
            dcc.Graph(id='output',config={'displayModeBar': False, 'autosizable':True, 'responsive':True},
                     style={'background-color':'#32383E', 'height':'100%'})
            
   
                
                
            ])
        ]),
        
        
        
         html.Br(),
        #--------------------------------3rd row Graphs----------------------------------------------#
        
        dbc.Row([
            
            #tabs
             dbc.Col([
                    dbc.Tabs(
                        [
                            dbc.Tab( label="Experience Ranges ", children=[drawFigure( figure=experience_fig)]),
                            dbc.Tab( label="Job type", children=[drawFigure( figure=job_type_bar)]),
                            dbc.Tab( label="career level", children=[drawFigure( figure=career_level_bar)]),
                            dbc.Tab( label="career level and experience", children=[drawFigure( figure=career_experience_bar)]),
                        ], style={'height':'15px'}
                    )
                ], width=3, style={'width':'40rem' , 'height':'15rem', 'position':'relative'}),
            html.Br(),
            
            # pie chart
            
            dbc.Col([
                drawFigure( figure=career_level_pie)
                
            ])
                              
            
    ] ),
    
          
          
          
          ]),
        
        
        
]),
    
])


In [304]:
@app.callback(
    Output(component_id='output',component_property="figure"),
    Input(component_id='input',component_property='value')

)
def update_Fig(value):
    figure=dcc.Graph()
    if value =='ALL':
        
        figure= skills_fig
    elif value =='AI':
        figure= AI_skills_fig
    elif value=='ML':
        figure= ML_skills_fig
    elif value=='DS':
        figure= DS_skills_fig
    elif value=='DA':
        figure= DA_skill_fig
    elif value=='DE':
        figure= DE_skills_fig
    
    return figure

In [310]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2022 15:15:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:05] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:05] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:05] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 15:15:24] "POST /_dash-update-component HTTP/1.